# Pipeline to Train and Test Networks

Paths & Library Imports

In [1]:
from os.path import isfile, exists
from paths import RETOUCH_PATH, IMAGES_PATH

Create Train-Test Split for the Fluid Segmentation Task

In [2]:
from init.foldsSplit import k_fold_split_segmentation

k = 5

if not (isfile(path="splits/segmentation_train_splits.csv") or isfile(path="splits/segmentation_test_splits.csv")):
    k_fold_split_segmentation(k=k, folders_path=RETOUCH_PATH)
else:
    print("Split already exists. To create a new one please delete the old files.")


Split already exists. To create a new one please delete the old files.


OCT Volumes Reading and Saving for Segmentation Task

In [ ]:
from init.readOCT import save_segmentation_oct_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\int32")) and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\uint8"))):
    save_segmentation_oct_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Images have already been extracted. To extract them again, please delete the folder with the images.")

OCT Masks Reading and Saving for Segmentation Task

In [ ]:
from init.readOCT import save_segmentation_mask_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\int32")) and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\masks\\uint8"))):
    save_segmentation_mask_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Masks have already been extracted. To extract them again, please delete the folder with the images.")

ROI Masks Extraction for Segmentation Task

In [5]:
from init.patchExtraction import extractROIMasks

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\int32") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\slices\\uint8"))):
    print("First, the images must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi")):
    extractROIMasks(oct_path=RETOUCH_PATH , folder_path=IMAGES_PATH, threshold=1e-2)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted ROI masks.")

Patches have already been extracted. To extract them again, please delete the folder that contains the extracted ROI masks.


Patches Extraction for 2D Segmentation Task

In [3]:
from init.patchExtraction import extractPatches

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\int8") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\uint8"))):
    print("First, the ROI masks must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\patches\\2D")):
    extractPatches(IMAGES_PATH, patch_shape=(256,128), n_pos=12, n_neg=2, pos=1, neg=0)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted patches.")

Patches have already been extracted. To extract them again, please delete the folder that contains the extracted patches.


Patches Extraction for 2.5D Segmentation Task

In [4]:
from init.patchExtraction import extractPatches25D

if not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\int8") and (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\roi\\uint8"))):
    print("First, the ROI masks must be extracted from the OCT volumes.")
elif not (exists(IMAGES_PATH + "\\OCT_images\\segmentation\\patches\\2.5D")):
    extractPatches25D(IMAGES_PATH, patch_shape=(256,128), n_pos=12, n_neg=2, pos=1, neg=0)
else:
    print("Patches have already been extracted. To extract them again, please delete the folder that contains the extracted patches.")

Patches have already been extracted. To extract them again, please delete the folder that contains the extracted patches.


Create Train-Test Split for the Intermediate Slice Generation Task

In [ ]:
from init.foldsSplit import k_fold_split_generation

k = 5

if not (isfile(path="splits/generation_train_splits.csv") or isfile(path="splits/generation_test_splits.csv")):
    k_fold_split_generation(k=k, folders_path=RETOUCH_PATH)
else:
    print("Split already exists. To create a new one please delete the old files.")

OCT Volumes Reading and Saving for Generation Task

Note: To make the generation task independent from the segmentation task, the images used for segmentation are being saved again in a different folder. For memory concerns, please adjust the code to reuse those saved previously.

In [ ]:
from init.readOCT import save_generation_oct_as_tiff

if not ((exists(IMAGES_PATH + "\\OCT_images\\generation\\int32")) and (exists(IMAGES_PATH + "\\OCT_images\\generation\\uint8"))):
    save_generation_oct_as_tiff(oct_folder=RETOUCH_PATH, save_folder=IMAGES_PATH)
else:
    print("Images have already been extracted. To extract them again, please delete the folder with the images.")